In [1]:
!pip3 install google-api-python-client

In [2]:
from apiclient.discovery import build
# from apiclient.errors import HttpError
# from oauth2client.tools import argparser

In [3]:
import json
with open('secret.json') as f:
    secret = json.load(f)

In [4]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

# def youtube_search(options):
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=DEVELOPER_KEY)

q = 'Python'
max_results = 50

response = youtube.search().list(
    q=q,
    part='id,snippet',
    order='viewCount',
    type='Video',
    maxResults=max_results
).execute()

In [5]:
items = response['items']
item = items[0]
item

{'kind': 'youtube#searchResult',
 'etag': 'pyb_xYIDKwmWX32RiZwksAyjc5I',
 'id': {'kind': 'youtube#video', 'videoId': 'xfYAj1k9uZM'},
 'snippet': {'publishedAt': '2009-07-26T13:20:15Z',
  'channelId': 'UCyNDOuol6OIKPImq-tmePZQ',
  'title': 'Python vs Alligator  01 -- Real Fight -- Python attacks Alligator',
  'description': 'Make A Donation: PayPal Website 01: http://www.ojatrovisuals.com/ Website 02: http://ojatro.com Facebook: ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/xfYAj1k9uZM/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/xfYAj1k9uZM/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/xfYAj1k9uZM/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'ojatro',
  'liveBroadcastContent': 'none',
  'publishTime': '2009-07-26T13:20:15Z'}}

In [6]:
video_id = item['id']['videoId']

In [7]:
channel_id = item['snippet']['channelId']

In [8]:
import pandas as pd

In [9]:
items = response['items']
items_id = []
for item in items:
    item_id = {}
    item_id['video_id'] = item['id']['videoId']
    item_id['channel_id'] = item['snippet']['channelId']
    items_id.append(item_id)
    
df_video = pd.DataFrame(items_id)

In [10]:
df_video[:3]

,video_id,channel_id
0,xfYAj1k9uZM,UCyNDOuol6OIKPImq-tmePZQ
1,dw5eILfy-c8,UCVTchywoHQniqChwOb66p0A
2,PU4ixyprX_Y,UCZiIqEDnYdJZXkUgJBKxIPA


In [11]:
def video_search(youtube, q='自動化', max_results=50):
    response = youtube.search().list(
        q=q,
        part='id,snippet',
        order='viewCount',
        type='Video',
        maxResults=max_results
    ).execute()

    items = response['items']
    items_id = []
    for item in items:
        item_id = {}
        item_id['video_id'] = item['id']['videoId']
        item_id['channel_id'] = item['snippet']['channelId']
        items_id.append(item_id)

    df_video = pd.DataFrame(items_id)
    
    return df_video

In [12]:
DEVELOPER_KEY = secret['KEY']
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                developerKey=DEVELOPER_KEY)

df_video = video_search(youtube, q='Python 自動化', max_results=50)

In [13]:
df_video[:3]

,video_id,channel_id
0,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ
1,-5v7CuXm7Ns,UCGlgXjYVoHLD86TQQ799WIw
2,vRFVS_FNLwY,UCwhKqH0jDKm5vPOZ7WQ2R1A


In [14]:
channel_ids = df_video['channel_id'].unique().tolist()
channel_ids[:3]

['UC0xRMqPOyRNPTaL6BxhbCnQ',
 'UCGlgXjYVoHLD86TQQ799WIw',
 'UCwhKqH0jDKm5vPOZ7WQ2R1A']

In [15]:
subscriber_list = youtube.channels().list(
    id =','.join(channel_ids),
    part='statistics',
    fields='items(id,statistics(subscriberCount))'
).execute()

In [16]:
subscriber_list['items'][:5]

[{'id': 'UCazV3A3_1-Mtd6E_auw_ifg',
  'statistics': {'subscriberCount': '18900'}},
 {'id': 'UC4iUfGeYbYhAySUwCZge9Ig', 'statistics': {}},
 {'id': 'UCrZOiJyMbXpZ8T5snCIJaLA',
  'statistics': {'subscriberCount': '93700'}},
 {'id': 'UC5Kgc_HNzx4GJ-w4QMeeKiQ',
  'statistics': {'subscriberCount': '17900'}},
 {'id': 'UC6M7Qk2It63qD0jhiKgLWZA',
  'statistics': {'subscriberCount': '34500'}}]

In [17]:
item_test = subscriber_list['items'][4]
print(item_test)
if 'subscriberCount' in item_test['statistics'].keys():
    print('ある')
else:
    print('ない')

{'id': 'UC6M7Qk2It63qD0jhiKgLWZA', 'statistics': {'subscriberCount': '34500'}}
ある


In [18]:
subscribers = []
for item in subscriber_list['items']:
    subscriber = {}
    subscriber['channel_id'] = item['id']
    if 'subscriberCount' in item['statistics'].keys():
        subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
    else:
        subscriber['subscriber_count'] = 0
    subscribers.append(subscriber)

df_subscribers = pd.DataFrame(subscribers)

In [19]:
df_subscribers[:5]

,channel_id,subscriber_count
0,UCazV3A3_1-Mtd6E_auw_ifg,18900
1,UC4iUfGeYbYhAySUwCZge9Ig,0
2,UCrZOiJyMbXpZ8T5snCIJaLA,93700
3,UC5Kgc_HNzx4GJ-w4QMeeKiQ,17900
4,UC6M7Qk2It63qD0jhiKgLWZA,34500


In [20]:
df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
df.head(3)

,video_id,channel_id,subscriber_count
0,bsq2YY-XAEo,UC0xRMqPOyRNPTaL6BxhbCnQ,111000
1,KmqEyT7HznQ,UC0xRMqPOyRNPTaL6BxhbCnQ,111000
2,VRFfAeW30qE,UC0xRMqPOyRNPTaL6BxhbCnQ,111000


In [21]:
df_extracted = df[df['subscriber_count'] < 20000]
df_extracted

,video_id,channel_id,subscriber_count
36,SPf_nINsCfw,UCGFxqqRaM97VjM5ToRfkoFg,6070
37,W3YNeebyplc,UCGFxqqRaM97VjM5ToRfkoFg,6070
38,nCfAtMooIQQ,UCNste53_VRtP6MGUfitOYQA,5810
41,bgzHqKN6teo,UCYPkI8lgfiNXmGxCglZhOwA,962
42,CDBcNB2RrvE,UCazV3A3_1-Mtd6E_auw_ifg,18900
44,1KOAwXTGpdo,UCHxqQ8bUg5F2GW79D_DvSjQ,15700
45,0zelr_2KkrU,UCaeuEZcPrQMdMvI6VMFHc5Q,4350
46,8xFdQ-0t12s,UC4iUfGeYbYhAySUwCZge9Ig,0
47,JP_ugJny9X4,UC5Kgc_HNzx4GJ-w4QMeeKiQ,17900


In [22]:
video_ids = df_extracted['video_id'].tolist()

In [23]:
video_ids

['SPf_nINsCfw',
 'W3YNeebyplc',
 'nCfAtMooIQQ',
 'bgzHqKN6teo',
 'CDBcNB2RrvE',
 '1KOAwXTGpdo',
 '0zelr_2KkrU',
 '8xFdQ-0t12s',
 'JP_ugJny9X4']

In [24]:
videos_list = youtube.videos().list(
    id =','.join(video_ids),
    part='snippet,statistics',
    fields='items(id,snippet(title),statistics(viewCount))'
).execute()

In [25]:
videos_list['items'][:3]

[{'id': 'SPf_nINsCfw',
  'snippet': {'title': '【2022年版】Pythonで業務自動化(Excel,Outlook等)したいならこの本で決定'},
  'statistics': {'viewCount': '72943'}},
 {'id': 'W3YNeebyplc',
  'snippet': {'title': '【PythonでExcel操作】オススメ解説本はこれ！【業務効率・自動化用】'},
  'statistics': {'viewCount': '12442'}},
 {'id': 'nCfAtMooIQQ',
  'snippet': {'title': 'その面倒な作業、自動化してみない？【Pythonで自動化のすすめ】'},
  'statistics': {'viewCount': '38134'}}]

In [26]:
videos_info = []
items = videos_list['items']
for item in items:
    video_info = {}
    video_info['video_id'] = item['id']
    video_info['title'] = item['snippet']['title']
    video_info['view_count'] = item['statistics']['viewCount']
    videos_info.append(video_info)
df_videos_info = pd.DataFrame(videos_info)

In [27]:
df_videos_info

,video_id,title,view_count
0,SPf_nINsCfw,"【2022年版】Pythonで業務自動化(Excel,Outlook等)したいならこの本で決定",72943
1,W3YNeebyplc,【PythonでExcel操作】オススメ解説本はこれ！【業務効率・自動化用】,12442
2,nCfAtMooIQQ,その面倒な作業、自動化してみない？【Pythonで自動化のすすめ】,38134
3,bgzHqKN6teo,面倒なExcel作業をPythonで自動化してみた【複数ファイルからデータを一瞬で抽出】,23260
4,CDBcNB2RrvE,"【Python】pytest - 自动化测试工具, 帮你写出最好的程序",19683
5,1KOAwXTGpdo,タイピングの練習がめんどくさすぎたからプログラムで自動化した結果ｗｗｗｗ,16998
6,0zelr_2KkrU,【Pythonで自動化？】Seleniumでブラウザを自動で動かしてみよう！,14824
7,8xFdQ-0t12s,Python自動化Excel數據演示，學習辦公必備技能。合併工作表，讀取特定數據，批量添加公...,14085
8,JP_ugJny9X4,【Pythonプログラミング実践編】Excel操作を自動化をしてみよう！〜初心者編〜,13127


In [28]:
results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
results[:3]

,video_id,channel_id,subscriber_count,title,view_count
0,SPf_nINsCfw,UCGFxqqRaM97VjM5ToRfkoFg,6070,"【2022年版】Pythonで業務自動化(Excel,Outlook等)したいならこの本で決定",72943
1,W3YNeebyplc,UCGFxqqRaM97VjM5ToRfkoFg,6070,【PythonでExcel操作】オススメ解説本はこれ！【業務効率・自動化用】,12442
2,nCfAtMooIQQ,UCNste53_VRtP6MGUfitOYQA,5810,その面倒な作業、自動化してみない？【Pythonで自動化のすすめ】,38134


In [29]:
results = results.loc[:,['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]

In [30]:
results

,video_id,title,view_count,subscriber_count,channel_id
0,SPf_nINsCfw,"【2022年版】Pythonで業務自動化(Excel,Outlook等)したいならこの本で決定",72943,6070,UCGFxqqRaM97VjM5ToRfkoFg
1,W3YNeebyplc,【PythonでExcel操作】オススメ解説本はこれ！【業務効率・自動化用】,12442,6070,UCGFxqqRaM97VjM5ToRfkoFg
2,nCfAtMooIQQ,その面倒な作業、自動化してみない？【Pythonで自動化のすすめ】,38134,5810,UCNste53_VRtP6MGUfitOYQA
3,bgzHqKN6teo,面倒なExcel作業をPythonで自動化してみた【複数ファイルからデータを一瞬で抽出】,23260,962,UCYPkI8lgfiNXmGxCglZhOwA
4,CDBcNB2RrvE,"【Python】pytest - 自动化测试工具, 帮你写出最好的程序",19683,18900,UCazV3A3_1-Mtd6E_auw_ifg
5,1KOAwXTGpdo,タイピングの練習がめんどくさすぎたからプログラムで自動化した結果ｗｗｗｗ,16998,15700,UCHxqQ8bUg5F2GW79D_DvSjQ
6,0zelr_2KkrU,【Pythonで自動化？】Seleniumでブラウザを自動で動かしてみよう！,14824,4350,UCaeuEZcPrQMdMvI6VMFHc5Q
7,8xFdQ-0t12s,Python自動化Excel數據演示，學習辦公必備技能。合併工作表，讀取特定數據，批量添加公...,14085,0,UC4iUfGeYbYhAySUwCZge9Ig
8,JP_ugJny9X4,【Pythonプログラミング実践編】Excel操作を自動化をしてみよう！〜初心者編〜,13127,17900,UC5Kgc_HNzx4GJ-w4QMeeKiQ


In [31]:
def get_results(df_video, threshold=20000):
    channel_ids = df_video['channel_id'].unique().tolist()

    subscriber_list = youtube.channels().list(
        id =','.join(channel_ids),
        part='statistics',
        fields='items(id,statistics(subscriberCount))'
    ).execute()

    subscribers = []
    for item in subscriber_list['items']:
        subscriber = {}
        if len(item['statistics']) > 0:
            subscriber['channel_id'] = item['id']
            subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
        else:
            subscriber['channel_id'] = item['id']
        subscribers.append(subscriber)
            
#         if 'subscriberCount' in item['statistics'].keys():
#             subscriber['subscriber_count'] = int(item['statistics']['subscriberCount'])
#         else:
#             subscriber['subscriber_count'] = 0

    df_subscribers = pd.DataFrame(subscribers)

    df = pd.merge(left=df_video, right=df_subscribers, on='channel_id')
    df_extracted = df[df['subscriber_count'] < 20000]

    video_ids = df_extracted['video_id'].tolist()
    videos_list = youtube.videos().list(
        id =','.join(video_ids),
        part='snippet,statistics',
        fields='items(id,snippet(title),statistics(viewCount))'
    ).execute()

    videos_info = []
    items = videos_list['items']
    for item in items:
        video_info = {}
        video_info['video_id'] = item['id']
        video_info['title'] = item['snippet']['title']
        video_info['view_count'] = item['statistics']['viewCount']
        videos_info.append(video_info)

    df_videos_info = pd.DataFrame(videos_info)

    results = pd.merge(left=df_extracted, right=df_videos_info, on='video_id')
    results = results.loc[:,['video_id', 'title', 'view_count', 'subscriber_count', 'channel_id']]
    return results

In [33]:
df_video = video_search(youtube, q='Python Excel', max_results=50)
results = get_results(df_video, threshold=5000)

In [34]:
results

,video_id,title,view_count,subscriber_count,channel_id
0,ubqsRcCUcB4,Use Python to replace VBA in Excel,333245,9050.0,UCfMaA6FC0aKYXEWubmOW5Vw
1,eWRtrPeJoyA,Automate Excel using Python | Excel Hacks with...,274164,16800.0,UCyVIfVgx77jslb0U5h-zhLQ
2,71zkSuzkJrw,How to Use SQL with Excel using Python,89653,16800.0,UCyVIfVgx77jslb0U5h-zhLQ
3,YwIX8w9i2DM,Automate Excel with Python Tutorial (2020),244382,6560.0,UCbg0ptFI3yZ4Pe5vmUiEzzA
4,hT4b9vFZRB0,Import Excel data file into python pandas : Re...,122028,10800.0,UCRhhFunXogiEK3WiinHGTAQ
5,SPf_nINsCfw,"【2022年版】Pythonで業務自動化(Excel,Outlook等)したいならこの本で決定",72943,6070.0,UCGFxqqRaM97VjM5ToRfkoFg
6,knnhkraHsBg,Xlsxwriter Python - Write Excel files with Python,72890,1660.0,UCxF1I4qFHT2qRGj1Dke4wBw
7,rQUYpJvMgn8,Python Pandas #1 - Lendo os dados do Excel,65431,17400.0,UCcAReLUkgigxmds2UfLCrgw
8,syEfR1QIGcY,[19] Convert a multi-page PDF file into csv / ...,59119,8060.0,UCQN09g3-sWVRDQc93WRZKYg
9,7zeAIEPJaoQ,Turn Your Excel File Into A Web App With Pytho...,52003,13700.0,UCZjRcM1ukeciMZ7_fvzsezQ
